Import all the relevant packages.

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from my_func import *

Define the architecture of the two neural networks we are going to compare.

In [12]:

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 1200)
        self.fc2 = nn.Linear(1200, 500)
        self.fc3 = nn.Linear(500, num_classes)

    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x       

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

Set device to cuda if it exists, otherwise run on cpu.

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device.type

'cuda'

Define input size(only used for linnear model), number of classes in the dataset, learning rate, batch size, number of epochs and the classes in the given dataset(for testing accuracy later).

In [14]:
input_size = 32*32*3
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 100
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Load/download the desired dataset. Split them into train and test sets.

In [15]:
train_dataset = datasets.CIFAR10(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.CIFAR10(root = 'dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


Create the models and define lossfunction and optimizer.

In [16]:
linear_model = NN(input_size = input_size, num_classes = num_classes).to(device = device)
convolutional_model = CNN().to(device = device)

criterion = nn.CrossEntropyLoss()

linear_optimizer = optim.Adam(linear_model.parameters(), lr = learning_rate)
convolutional_optimizer = optim.Adam(convolutional_model.parameters(), lr = learning_rate)

Train both models using their forward function and my own train function(see my_func.py).

In [17]:
my_train(linear_model, num_epochs, train_loader, criterion, linear_optimizer, device) #Train Linear model
my_train(convolutional_model, num_epochs, train_loader, criterion, convolutional_optimizer, device) #Train convolutional model

Check overall accuracy on train and test sets with both models.

In [18]:
print("Linear model: ")
check_accuracy(train_loader, linear_model, device)
check_accuracy(test_loader, linear_model, device)
print("Convolutional model: ")
check_accuracy(train_loader, convolutional_model, device)
check_accuracy(test_loader, convolutional_model, device)

Linear model: 
Checking accuracy on the training data
Got 14349 / 50000 with accuracy 28.70
Checking accuracy on test data
Got 2829 / 10000 with accuracy 28.29
Convolutional model: 
Checking accuracy on the training data
Got 24390 / 50000 with accuracy 48.78
Checking accuracy on test data
Got 4700 / 10000 with accuracy 47.00


Check the accuracy of the models based on specific classes in the datasets.

In [19]:
print("Checking accuracy of convolutional model: \n")
check_accuracy_by_class(classes, test_loader, device, linear_model)
print("Number of parameters in Linear NN:", sum(p.numel() for p in linear_model.parameters() if p.requires_grad))

Checking accuracy of convolutional model: 

Accuracy for class: plane is 11.1 %
Accuracy for class: car   is 10.5 %
Accuracy for class: bird  is 0.0 %
Accuracy for class: cat   is 2.2 %
Accuracy for class: deer  is 15.0 %
Accuracy for class: dog   is 43.5 %
Accuracy for class: frog  is 68.6 %
Accuracy for class: horse is 22.2 %
Accuracy for class: ship  is 39.6 %
Accuracy for class: truck is 70.2 %
Number of parameters in Linear NN: 4293110


In [20]:
print("Checking accuracy of convolutional model: \n")
check_accuracy_by_class(classes, test_loader, device, convolutional_model)
print("Number of parameters in convolutional NN:", sum(p.numel() for p in convolutional_model.parameters() if p.requires_grad))

Checking accuracy of convolutional model: 

Accuracy for class: plane is 58.6 %
Accuracy for class: car   is 73.7 %
Accuracy for class: bird  is 23.5 %
Accuracy for class: cat   is 27.9 %
Accuracy for class: deer  is 31.2 %
Accuracy for class: dog   is 41.3 %
Accuracy for class: frog  is 79.6 %
Accuracy for class: horse is 37.7 %
Accuracy for class: ship  is 48.7 %
Accuracy for class: truck is 47.8 %
Number of parameters in convolutional NN: 61718
